### Import Libraries


In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests

### Assign web site adress to a variable
### use request and Beautiful Soup libraries to get text

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page,'xml')

In [3]:
table = soup.find('table')


### Create Lists for each columns

In [4]:
postcode = []
borough = []
neighbourhood = []

### use for-loop to pick values from the text and asign to the list

In [5]:
for tr_cell in table.find_all('tr'):
    counter = 1
    postcode_var = -1
    borough_var = -1
    neighbourhood_var = -1
    for td_cell in tr_cell.find_all('td'):
        if counter == 1:
            postcode_var = td_cell.text
        if counter == 2:
            borough_var = td_cell.text
            tag_a_barough = td_cell.find('a')
        if counter == 3:
            neighbourhood_var = str(td_cell.text).strip()
            tag_a_neighbourhood = td_cell.find('a')
        counter +=1
        if (postcode_var == 'Not assigned' or borough_var == 'Not assigned' or neighbourhood_var == 'Not assigned'):
            continue
    try:
        if ((tag_a_barough is None)) or (tag_a_neighbourhood is None):
            continue
    except:
        pass
    if(postcode_var == -1 or borough_var == -1 or neighbourhood_var == -1):
        continue
    
    postcode.append(postcode_var)
    borough.append(borough_var)
    neighbourhood.append(neighbourhood_var)



In [6]:
unique_code = set(postcode)
postcode_u = []
borough_u = []
neighbourhood_u = []

for pc_u in unique_code:
    p = ''; b = ''; n='';
    for postcode_idx, postcode_element in enumerate(postcode):
        if pc_u == postcode_element:
            p = postcode_element;
            b = borough[postcode_idx]
            if n == '':
                n = neighbourhood[postcode_idx]
            else:
                n = n + ', ' + neighbourhood[postcode_idx]
    postcode_u.append(p)
    borough_u.append(b)
    neighbourhood_u.append(n)

### Create a table adding all columns together 

In [7]:
toronto = {'postcode':postcode_u,'borough':borough_u,'neighbourhood':neighbourhood_u}
df_canada = pd.DataFrame.from_dict(toronto)


In [8]:
df_canada.head(10)

,postcode,borough,neighbourhood
0,M6S,West Toronto,"Runnymede, Swansea"
1,M4Y,Downtown Toronto,Church and Wellesley
2,M5R,Central Toronto,"The Annex, Yorkville"
3,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
4,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
5,M9R,Etobicoke,Kingsview Village
6,M2L,North York,York Mills
7,M1V,Scarborough,"Agincourt North, Milliken"
8,M5L,Downtown Toronto,Commerce Court
9,M1K,Scarborough,"Ionview, Kennedy Park"


### Get coordinates from the .csv file 

In [9]:
df_coord = pd.read_csv('./Geospatial_Coordinates.csv')

In [10]:
df_coord.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_loc = df_coord.rename(columns={"Postal Code": "postcode"})

In [12]:
df_loc.head(5)

,postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_canada.head(5)

,postcode,borough,neighbourhood
0,M6S,West Toronto,"Runnymede, Swansea"
1,M4Y,Downtown Toronto,Church and Wellesley
2,M5R,Central Toronto,"The Annex, Yorkville"
3,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
4,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"


### Combine coordinates to the correspondong postcodes

In [14]:
result = pd.merge(df_canada, df_loc, on='postcode')

In [15]:
result.head(5)

,postcode,borough,neighbourhood,Latitude,Longitude
0,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
1,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
2,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678
3,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",43.686412,-79.400049
4,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049


### Use folium to generate map

In [16]:
import folium

In [17]:
map_Toranto = folium.Map(location=["43.6532", "-79.3832"], zoom_start=11)

In [18]:
for lat, lng, label in zip(result['Latitude'], result['Longitude'], result['neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toranto)  



In [19]:
map_Toranto